# Laboratorio 8 - Word Embeddings
## <font size=4> *CC3092 - Deep Learning y Sistemas Inteligentes*, 2024 </font>
## <font size=3 color='gray'> Samuel Chamale, Adrian Rodriguez y Daniel Gómez</font>

### <ins>1. (Modelo de Transporte)</ins>

In [23]:
using JuMP, GLPK, Printf

function solve_transportation_model(centros, concesionarios, distancias, ofertas, demandas)
    # Crear el modelo
    modelo = Model(GLPK.Optimizer)

    num_centros = length(centros)
    num_concesionarios = length(concesionarios)

    # Definir variables de decisión
    @variable(modelo, x[1:num_centros, 1:num_concesionarios] >= 0)

    # Definir la función objetivo (minimizar el costo total de transporte)
    @objective(modelo, Min,
        sum(distancias[i][j] * x[i,j] for i in 1:num_centros, j in 1:num_concesionarios) * (25/18)
    )

    # Restricciones de Oferta
    for i in 1:num_centros
        @constraint(modelo, sum(x[i,j] for j in 1:num_concesionarios) == ofertas[i])
    end

    # Restricciones de Demanda
    for j in 1:num_concesionarios
        @constraint(modelo, sum(x[i,j] for i in 1:num_centros) == demandas[j])
    end

    # Resolver el modelo
    optimize!(modelo)

    # Verificar el estado de la optimización
    println("\nResultados:\n")

    # Resultados en formato de tabla
    # Imprimir encabezado de concesionarios
    print(" " ^ 15)  # Espacio en blanco para la primera columna
    for j in 1:5
        print(rpad(concesionarios[j], 20))  # Imprimir cada concesionario con un ancho fijo
    end
    println()

    # Imprimir los valores de transporte con centros como filas
    for i in 1:3
        print(rpad(centros[i], 15))  # Imprimir el nombre del centro
        for j in 1:5
            print(rpad(@sprintf("%.2f", value(x[i,j])), 20))  # Imprimir el valor del transporte con 2 decimales
        end
        println()
    end

    @printf("\nCosto Total de Transporte: \$%.2f\n", objective_value(modelo))

    if termination_status(modelo) == MOI.OPTIMAL
        println("\nSe encontró una solución óptima.")
    else
        println("\nNo se encontró una solución óptima.")
    end
end

# Ejemplo de uso
centros = ["Cen 1", "Cen 2", "Cen 3"]
concesionarios = ["Con 1", "Con 2", "Con 3", "Con 4", "Con 5"]
distancias = [
    [100, 150, 200, 140, 35],
    [50, 70, 60, 65, 80],
    [40, 90, 100, 150, 130]
]
ofertas = [400, 200, 150]
demandas = [100, 200, 150, 160, 140]

solve_transportation_model(centros, concesionarios, distancias, ofertas, demandas)



Resultados:

               Con 1               Con 2               Con 3               Con 4               Con 5               
Cen 1          0.00                100.00              0.00                160.00              140.00              
Cen 2          0.00                50.00               150.00              0.00                0.00                
Cen 3          100.00              50.00               0.00                0.00                0.00                

Costo Total de Transporte: $87916.67

Se encontró una solución óptima.


### <ins>2. (Modelo de Transporte) - Variaciones</ins>

#### ***a) Concesionario 5 requiere 200 automóviles***

In [24]:
centros = ["Cen 1", "Cen 2", "Cen 3"]
concesionarios = ["Con 1", "Con 2", "Con 3", "Con 4", "Con 5"]
distancias = [
    [100, 150, 200, 140, 35],
    [50, 70, 60, 65, 80],
    [40, 90, 100, 150, 130]
]
ofertas = [400, 200, 150]
demandas = [100, 200, 150, 160, 200]

solve_transportation_model(centros, concesionarios, distancias, ofertas, demandas)


Resultados:

               Con 1               Con 2               Con 3               Con 4               Con 5               
Cen 1          0.00                0.00                0.00                260.00              140.00              
Cen 2          0.00                150.00              150.00              -100.00             0.00                
Cen 3          100.00              50.00               0.00                0.00                0.00                

Costo Total de Transporte: $87222.22

No se encontró una solución óptima.


#### ***b) Centro 3 produce 200 automóviles***

In [21]:
# Datos
centros = ["Cen 1", "Cen 2", "Cen 3"]
concesionarios = ["Con 1", "Con 2", "Con 3", "Con 4", "Con 5"]
distancias = [
    [100, 150, 200, 140, 35],
    [50, 70, 60, 65, 80],
    [40, 90, 100, 150, 130]
]
ofertas = [400, 200, 200]
demandas = [100, 200, 150, 160, 140]

solve_transportation_model(centros, concesionarios, distancias, ofertas, demandas)


Resultados:

               Con 1               Con 2               Con 3               Con 4               Con 5               
Cen 1          0.00                0.00                0.00                260.00              140.00              
Cen 2          0.00                150.00              150.00              -100.00             0.00                
Cen 3          100.00              50.00               0.00                0.00                0.00                

Costo Total de Transporte: $87222.22
No se encontró una solución óptima.
